In [533]:
import pandas as pd 
import torch 
from torch import nn
from sklearn.preprocessing import OneHotEncoder,StandardScaler
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score



In [ ]:
data = pd.read_csv(".//diabetes.csv")
data.head()

In [535]:
x = data.drop(columns=['Class variable']).to_numpy()
y = data[['Class variable' ]].to_numpy()

In [536]:
sc = StandardScaler()
x = sc.fit_transform(x)

In [537]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=69)

In [538]:
## train data
class TrainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = TrainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))
                  
## test data    
class TestData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = TestData(torch.FloatTensor(X_test))

In [539]:
EPOCHS = 20
BATCH_SIZE = 4
LEARNING_RATE = 0.01

In [540]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [541]:
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        # Number of input features is 8.
        self.layer_1 = nn.Linear(8, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_out = nn.Linear(64, 1) 
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        return x

In [542]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [543]:
model = BinaryClassification()
model.to(device)
print(model)

BinaryClassification(
  (layer_1): Linear(in_features=8, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [544]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [545]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [546]:
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = loss_fn(y_pred, y_batch)
        acc = binary_acc(y_pred, y_batch)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 001: | Loss: 0.60009 | Acc: 66.572
Epoch 002: | Loss: 0.55364 | Acc: 72.931
Epoch 003: | Loss: 0.54813 | Acc: 69.942
Epoch 004: | Loss: 0.56667 | Acc: 72.064
Epoch 005: | Loss: 0.54752 | Acc: 72.064
Epoch 006: | Loss: 0.53850 | Acc: 72.642
Epoch 007: | Loss: 0.52549 | Acc: 73.555
Epoch 008: | Loss: 0.53455 | Acc: 72.208
Epoch 009: | Loss: 0.54773 | Acc: 73.746
Epoch 010: | Loss: 0.52194 | Acc: 74.376
Epoch 011: | Loss: 0.52481 | Acc: 74.520
Epoch 012: | Loss: 0.54658 | Acc: 74.231
Epoch 013: | Loss: 0.53255 | Acc: 71.919
Epoch 014: | Loss: 0.52309 | Acc: 74.954
Epoch 015: | Loss: 0.53051 | Acc: 75.098
Epoch 016: | Loss: 0.53571 | Acc: 73.555
Epoch 017: | Loss: 0.52542 | Acc: 74.809
Epoch 018: | Loss: 0.54646 | Acc: 72.832
Epoch 019: | Loss: 0.50602 | Acc: 76.012
Epoch 020: | Loss: 0.53535 | Acc: 73.220


In [547]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [548]:
confusion_matrix(y_test, y_pred_list)

array([[45,  1],
       [20, 11]])

In [549]:
accuracy_score(y_test,y_pred_list)

0.7272727272727273

In [550]:
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

           0       0.69      0.98      0.81        46
           1       0.92      0.35      0.51        31

    accuracy                           0.73        77
   macro avg       0.80      0.67      0.66        77
weighted avg       0.78      0.73      0.69        77

